In [1]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from tqdm import tqdm

### Node & Monte Carlo Tree Search Classes



In [2]:
class Node():
  def __init__(self, board, parent_node):
    self.board = board
    self.is_terminal = self.board.win() or self.board.draw()
    self.parent = parent_node
    self.total_returns = 0
    self.number_action_taken = 0
    self.expanded = self.is_terminal
    self.children_nodes = {}

class MCTS():
  def search(self, starting_state, num_iterations):
    self.root = Node(starting_state, None)

    for iter in range(num_iterations):
      node = self.selection(self.root)
      total_returns = self.simulation(node.board)
      self.backpropagation(node, total_returns)
    
    try:
      return self.get_max_action(self.root, 0)
    except:
      pass
  
  def selection(self, node):
    while not node.is_terminal:
      if node.expanded:
        node = self.get_max_action(node, 2)
      else:
        return self.expansion(node)
      
    return node
  
  def expansion(self, node):
    actions = node.board.get_valid_actions()
    for act in actions:
      if str(act.board_positions) not in node.children_nodes:
        new_node = Node(act, node)

        node.children_nodes[str(act.board_positions)] = new_node

        if len(actions) == len(node.children_nodes):
          node.expanded = True
        
        return new_node
  
  def get_max_action(self, node, exploration_constant):
    max_return = float('-inf')
    max_actions = []

    for child in node.children_nodes.values():
      curr_player = None
      if child.board.second_player == 'X': curr_player = 1
      elif child.board.second_player == 'O': curr_player = -1

      UCB_val = curr_player * child.total_returns/child.number_action_taken + np.sqrt((exploration_constant * np.log(node.number_action_taken))/child.number_action_taken)
      if UCB_val > max_return:
        max_return = UCB_val
        max_actions = [child]
      
      elif UCB_val == max_return:
        max_actions.append(child)

    return random.choice(max_actions)

  def simulation(self, board):
    while not board.win():
      try:
        board = random.choice(board.get_valid_actions())
      except:
        return 0
    
    if board.second_player == 'X': return 1
    if board.second_player == 'O': return -1
  
  def backpropagation(self, node, total_returns):
    while node:
      node.number_action_taken += 1
      node.total_returns += total_returns
      node = node.parent


###Tic-Tac-Toe Board Environment

In [3]:
class TicTacToeBoardEnvironment():
  def __init__(self, num_rows, num_cols, prev_board=None):
    self.first_player = 'X'
    self.second_player = 'O'
    self.curr_player = self.first_player
    self.empty = '*'
    self.num_rows = num_rows
    self.num_cols = num_cols
    self.board_positions = {}
      
    self.initialize_board()

    if prev_board:
      self.__dict__ = deepcopy(prev_board.__dict__)

  def reset(self):
    new_board = TicTacToeBoardEnvironment(self.num_rows, self.num_cols)
    terminated = False
    return new_board, terminated

  def print_board(self):
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        print(self.board_positions[row, col], end = " ")
      print()
    
    self.print_player()
  
  def print_player(self):
    print("\n" + self.curr_player + " please make a move...\n")

  def get_valid_actions(self):
    actions = []
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        if self.board_positions[row, col] == self.empty:
          actions.append(self.choose_position(row,col)) 
    
    return actions
  
  def get_valid_actions_(self):
    actions = []
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        if self.board_positions[row, col] == self.empty:
          actions.append((row,col)) 
    
    return actions

  def initialize_board(self):
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        self.board_positions[row, col] = self.empty
  
  def choose_position(self, row, col):
    new_board = TicTacToeBoardEnvironment(self.num_rows, self.num_cols, prev_board=self)
    new_board.board_positions[row, col] = self.first_player
    new_board.first_player, new_board.second_player = new_board.second_player, new_board.first_player
    new_board.curr_player = new_board.first_player

    return new_board
  
  def draw(self):
    for row in range(self.num_rows):
      for col in range(self.num_cols):
        if self.board_positions[row, col] == self.empty:
          return False
    return True
  
  def win(self):
    for row in range(self.num_rows):
      num_curr_player = []
      for col in range(self.num_cols):
        if self.board_positions[row, col] == self.second_player:
          num_curr_player.append((row, col))
        if len(num_curr_player) == self.num_rows:
          return True
          
    for col in range(self.num_cols):
      num_curr_player = []
      for row in range(self.num_rows):
        if self.board_positions[row, col] == self.second_player:
          num_curr_player.append((row, col))
        if len(num_curr_player) == self.num_rows:
          return True

    num_curr_player = []
    for row in range(self.num_rows):
      col = row
      if self.board_positions[row, col] == self.second_player:
        num_curr_player.append((row,col))
      if len(num_curr_player) == self.num_rows:
        return True

    num_curr_player = []
    for row in range(self.num_rows):
      col = self.num_rows - row - 1
      if self.board_positions[row, col] == self.second_player:
        num_curr_player.append((row,col))
      if len(num_curr_player) == self.num_rows:
        return True

    return False
    
  def terminal(self):
    if self.win():
      return 1 if self.second_player == 'X' else -1
    
    if self.draw():
      return 0
    
    return None

  def step(self, action):
    row, col = action
    self = self.choose_position(row, col)
    terminal_ = self.terminal()
    if terminal_ is not None:
      if terminal_ == 1:
        return TicTacToeBoardEnvironment(self.num_rows, self.num_cols), 100, True, "win"
      if terminal_ == 0:
        return TicTacToeBoardEnvironment(self.num_rows, self.num_cols), 0, True, "draw"
    
    rnd_idx = np.random.choice(len(self.get_valid_actions_()))
    row,col = self.get_valid_actions_()[rnd_idx]
    self = self.choose_position(row, col)

    terminal_ = self.terminal()
    if terminal_ is not None:
      if terminal_ == -1:
        return TicTacToeBoardEnvironment(self.num_rows, self.num_cols), -100, True, "loss"
      if terminal_ == 0:
        return TicTacToeBoardEnvironment(self.num_rows, self.num_cols), 0, True, "draw"

    return self, -1, False, "in-progress"


  def play_game_random(self, num_planning_steps):
    #print("Playing nxn Tic-Tac-Toe\n")
    #self.print_board()

    mcts = MCTS()
    while True:
      rnd_idx = np.random.choice(len(self.get_valid_actions_()))
      row,col = self.get_valid_actions_()[rnd_idx]
      #random.randint(0, self.num_rows-1)
      #col = random.randint(0, self.num_cols-1)

      while self.board_positions[row, col] != self.empty:
        rnd_idx = np.random.choice(len(self.get_valid_actions_()))
        row,col = self.get_valid_actions_()[rnd_idx]

      self = self.choose_position(row, col)

      max_action = mcts.search(self, num_planning_steps)
      try:
        self = max_action.board
      except:
        pass
      #self.print_board()

      terminal_ = self.terminal()
      if terminal_ is not None: return terminal_



###100 Games of Tic-Tac-Toe - MCTS(25,50,100 planning-steps) vs Random Opponent[board-sizes 3x3-10x10]


In [29]:
def mcts_vs_random_results(planning_steps):
  results = {"3x3": (), "4x4": (), "5x5": (), "6x6": (), "7x7": (), "8x8": (), "9x9": (), "10x10": ()}

  for board_size in range(3, 4):
    board = TicTacToeBoardEnvironment(board_size,board_size)
    wins_mcts, wins_random, draws = 0,0,0
    for i in tqdm(range(100)):
      res = board.play_game_random(planning_steps)
      if res == -1:
        wins_mcts += 1
      if res == 1:
        wins_random += 1
      if res == 0:
        draws += 1
    results[str(board_size) + "x" + str(board_size)] = (wins_mcts, wins_random, draws)

  return results


In [ ]:
mcts_vs_random_results_25_planning_steps = mcts_vs_random_results(25)

In [ ]:
mcts_vs_random_results_50_planning_steps = mcts_vs_random_results(50)

In [ ]:
mcts_vs_random_results_100_planning_steps = mcts_vs_random_results(100)

In [ ]:
# 25 Iterations
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_random_results_25_planning_steps
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('MCTS wins against Random Agent for Various NxN sizes of boards over 100 games(simulations=25)')

plt.show()

In [ ]:
# 50 Iterations
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_random_results_50_planning_steps

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('MCTS wins against Random Agent for Various NxN sizes of boards over 100 games(simulations=50)')

plt.show()

In [ ]:
# 100 Iterations
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_random_results_100_planning_steps
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('MCTS wins against Random Agent for Various NxN sizes of boards over 100 games(simulations=100)')

plt.show()

###100 Games of Tic-Tac-Toe - MCTS(25,50,100 planning-steps) vs Itself[board-sizes 3x3-10x10]

In [19]:
def mcts_vs_mcts_results(planning_steps):
  results = {"3x3": (), "4x4": (), "5x5": (), "6x6": (), "7x7": (), "8x8": (), "9x9": (), "10x10": ()}

  for board_size in range(3, 7):
    mcts = MCTS()
    count_wins = 0
    count_draws = 0

    for i in tqdm(range(100)):
      msg = ""
      board = TicTacToeBoardEnvironment(board_size, board_size)
      while msg != "invalid":
        max_act = mcts.search(board, planning_steps)
        try:
          board = max_act.board
        except: 
          msg = "invalid"
        #board.print_board()
        if board.win():
          count_wins += 1
          msg = "invalid"

        if board.draw():
          count_draws += 1
          msg = "invalid"
    results[str(board_size) + "x" + str(board_size)] = (count_wins, count_draws)
  return results

In [ ]:
mcts_vs_mcts_results_25_planning_steps = mcts_vs_mcts_results(25)

In [ ]:
mcts_vs_mcts_results_50_planning_steps = mcts_vs_mcts_results(50)

In [ ]:
mcts_vs_mcts_results_100_planning_steps = mcts_vs_mcts_results(100)

In [ ]:
# 25 Iterations
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_mcts_results_25_planning_steps

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('MCTS wins against itself for Various NxN sizes of boards over 100 games(simulations=25)')

plt.show()

In [ ]:
# 50 Iterations
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_mcts_results_50_planning_steps

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('Agent wins against itself for Various NxN sizes of boards over 100 games(simulations=50)')

plt.show()

In [ ]:
# 100 Iterations

objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = mcts_vs_mcts_results_100_planning_steps

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('Agent wins against itself for Various NxN sizes of boards over 100 games(simulations=100)')

plt.show()

### SARSA & Q-learning 
on Tic-Tac-Toe

In [4]:
import random
from collections import defaultdict
from tqdm import tqdm

def SARSA(env, alpha, gamma, epsilon, num_episodes, final_games_num):
  def policy(Q_vals, state, epsilon):
    rand_ = np.random.rand()
    valid_actions = state.get_valid_actions_()
    if rand_ > epsilon:
      act = np.argmax([Q_vals[(state,action)] for action in range(len(valid_actions))])
      return valid_actions[act]
    act = np.random.choice(len(valid_actions))
    return valid_actions[act]

  Q_vals = defaultdict(float)
  final_games = {"win": 0, "loss": 0, "draw": 0}

  for ep in tqdm(range(num_episodes)):
    state, _ = env.reset()
    action = policy(Q_vals, state, epsilon)
    flag = False
    while True:
      next_state, reward, done, status = state.step(action)
      next_action = policy(Q_vals, next_state, epsilon)
      Q_vals[(state,action)] = Q_vals[(state,action)] + alpha * (reward + gamma * Q_vals[(next_state,next_action)] - Q_vals[(state,action)])
      state,action = next_state,next_action
      if done:
        break
    if ep + final_games_num >= num_episodes:
      final_games[status] += 1
    
  return final_games

In [5]:
def SARSA_vs_random_results():
  results = {"3x3": (), "4x4": (), "5x5": (), "6x6": (), "7x7": (), "8x8": (), "9x9": (), "10x10": ()}
  for board_size in range(3, 11):
    final_games = SARSA(TicTacToeBoardEnvironment(board_size, board_size), 0.5, 0.9, 0.1, 1000, 100)
    results[str(board_size) + "x" + str(board_size)] = final_games['win']
  return results

In [ ]:
sarsa_results = SARSA_vs_random_results()

In [ ]:
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = list(sarsa_results.values())

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('SARSA wins against Random Agent for Various NxN sizes of boards over 100 games(num_episodes=1000)')

plt.show()

In [25]:
import random
from collections import defaultdict
from tqdm import tqdm

def Q_learning(env, alpha, gamma, epsilon, num_episodes, final_games_num):
  def policy(Q_vals, state, epsilon):
    rand_ = np.random.rand()
    valid_actions = state.get_valid_actions_()
    if rand_ > epsilon:
      act = np.argmax([Q_vals[(state,action)] for action in range(len(valid_actions))])
      return valid_actions[act]
    act = np.random.choice(len(valid_actions))
    return valid_actions[act]

  Q_vals = defaultdict(float)
  final_games = {"win": 0, "loss": 0, "draw": 0}

  for ep in tqdm(range(num_episodes)):
    state, _ = env.reset()
    while True:
      action = policy(Q_vals, state, epsilon)
      next_state, reward, done, status = state.step(action)
      next_state_valid_actions = next_state.get_valid_actions_()
      Q_MAX = np.max([Q_vals[(next_state, act)] for act in next_state_valid_actions])
      Q_vals[(state, action)] = Q_vals[(state, action)] + alpha * (reward + gamma * Q_MAX - Q_vals[(state, action)])
      state = next_state
      if done:
        break
    if ep + final_games_num >= num_episodes:
      final_games[status] += 1

  return final_games

In [26]:
def Q_learning_vs_random_results():
  results = {"3x3": (), "4x4": (), "5x5": (), "6x6": (), "7x7": (), "8x8": (), "9x9": (), "10x10": ()}
  for board_size in range(3, 11):
    final_games = Q_learning(TicTacToeBoardEnvironment(board_size, board_size), 0.5, 0.9, 0.1, 1000, 100)
    results[str(board_size) + "x" + str(board_size)] = final_games['win']
  return results

In [ ]:
q_learning_results = Q_learning_vs_random_results()

In [ ]:
objects = ('3x3', '4x4', '5x5', '6x6', '7x7', '8x8', '9x9', '10x10')
y_pos = np.arange(len(objects))
performance = list(q_learning_results.values())

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Number of Wins')
plt.ylabel('Board Size')

plt.title('Q-learning wins against Random Agent for Various NxN sizes of boards over 100 games(num_episodes=1000)')

plt.show()